In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Libraries and functions

In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
# import matplotlib as mpl
import sys
import time
import math
from tqdm import tqdm
from importlib import reload
from os.path import join, normpath
from bs4 import BeautifulSoup
import os
from os import path
import roamability as rb
import re

downloads = 'C:/Users/balob/Downloads'

In [99]:
%reset -f

# Examples

### MS SQL request

In [208]:
sql_srt='SELECT COUNT(*) FROM ACCOUNTS;'
with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

### DMI Oracle request

In [211]:
sql_srt = 'SELECT * FROM SPONSOR'
# with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI_TEST') as cnxn:
with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

### BSS MySql request

In [1096]:
sql_srt='SELECT MSISDN, VisitedNetworkTadig FROM TAP.GPRS_CALL LIMIT 5'
with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn)
df

,MSISDN,VisitedNetworkTadig
0,447700095423,BRAV1
1,972541033641,BRAV1
2,972541039874,BRAV1
3,972541033641,BRAV1
4,447700095423,BRAV1


### Soap API OCS Portal

In [163]:
# %reset -f
from zeep import Client
from zeep.wsse.username import UsernameToken

connect = input("Choose connection (prod or lab):")

if connect == 'lab':
    user = 'lab_api@roamability.com'
    user = 'api_lab@dialoq.com'
    password = '25D55AD283AA400AF464C76D713C07AD' #Lab
    password = 'FAC4EC08B5FC0CCF8145BB79E8F24F47' #Lab
    api_link = 'https://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
    api_link = 'https://labocsapi.roamability.com:6443/ocsapi/roamability/api/roamability.wsdl'
    api_link = 'http://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD'
    api_link = 'https://ocsapi.roamability.com:7443/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'https://ocsapi.roamability.com/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'https://ocsapi.roamability.com:7443/ocsapi/roamability/api/'
#     api_link = 'https://ocsapi2.roamability.com:7443/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'http://172.20.35.10:8585/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'http://172.20.35.12:8585/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))

# Subscriber in lab in STI account
#subscriber_id = client.service.getSubscriberById('934420')
#subscriber_iccid = client.service.getSubscriberByICCID('8997219121000031446')
# client.service.getSubscriberPrePaidPackages('831882', 1)
#client.service.getSubscriberByIMSI('425019613023160')
#client.service.getSubscriberById('209287')
#client.service.getSubscriberByICCID('8997219121000022660')
# client.service.assignSubscriberPrePaidPackage('831882', 1, dt.datetime.strptime('2019-07-07','%Y-%m-%d'))
# client.service.getSubscriberPrePaidPackages('937939', 1)
# client.service.expireSubscriberPrePaidPackage(1848972,60311)
# client.service.getSubscriberPrePaidPackages(1848972,1)
# prices = client.service.getMobilePrices() # prices.destinations[0]
# client.service.getPricingPlans()
# client.service.modifySubscriberSpeedLimit(1449606, '256kbps')
# client.service.modifySubscriberSpeedLimit(210000, '1024kbps') # lab
#subscriber_id

Choose connection (prod or lab):prod
You are connected to Reseller: Dialoq - Genesisclub RT.    
Available credit for the Reseller: 8085 USD.

The Reseller contains the following accounts:
 0    Genesisclub Test sim cards (accountID: 352475)
 1    Genesisclub Partner Soft test (accountID: 352642)
 2    Genesisclub P4 Soft test (accountID: 352643)
 3    Truphone S1 Soft (accountID: 352770)
 4    Truphone S2 Soft (accountID: 352771)
 5    Truphone QR DATA S2+S1+S4 (accountID: 352820)
 6    Dailoq S2+S1+S4 (accountID: 352827)
 7    New Profiles (accountID: 352857)
 8    new applet profile test (accountID: 352858)
 9    Truphone QR DATA_CALLS S2+S1+S4 (accountID: 352859)
 10   dialoq_10K_191119 (accountID: 352891)
 11   Truphone S5 Soft (accountID: 352954)
 12   Truphone S8 Soft (accountID: 352955)
 13   Dailoq_200522_5K_S2_S1 (accountID: 352979)


### Send MSU

In [679]:
reload(rb)
imsi = 260036610100002
# imsi = 260060169326039
orealm = 'epc.mnc003.mcc260.3gppnetwork.org'
ohost = 'nakahss.epc.mnc003.mcc260.3gppnetwork.org'

drealm = 'epc.mnc001.mcc231.3gppnetwork.org'
dhost = 'mme-01-ba3.epc.mnc001.mcc231.3gppnetwork.org'

mcc = '231'
mnc = '01'
soup = BeautifulSoup(
#     rb.air(imsi, ohost, orealm, drealm, mcc, mnc)
    rb.clr(imsi, ohost, orealm, dhost, drealm)
    ,'xml')
soup

<?xml version="1.0" encoding="utf-8"?>
<cla>
<session-id>cb0c8d67c1580b433a17231c7d137698ba0945de6baa9ed072b24b1239be2559;1591183832;8978434</session-id>
<orig-realm>epc.mnc001.mcc231.3gppnetwork.org</orig-realm>
<orig-host>draus01.comfone.com</orig-host>
<res-code>3002</res-code>
</cla>

 ### Format output

In [ ]:
import datetime as dt
print( 'start: {} {} :end'.format('one', 'two') ) #start: one two :end
print( 'start: {1} {0} :end'.format('one', 'two') ) #start: two one :end
print( 'start: test {:10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:10} :end'.format(10) ) #start: test         10 :end
print( 'start: test {:>10} :end'.format('test') ) #start: test       test :end
print( 'start: test {:<10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:_<10} :end'.format('test') ) #start: test test______ :end
print( 'start: test {:^10} :end'.format('test') ) #start: test    test    :end
print( 'start: test {:.10} :end'.format('test1test2test3') ) #start: test test1test2 :end
print( 'start: test {:_^10.5} :end'.format('test1test2test3') ) #start: test __test1___ :end
print( 'start: test {:d} :end'.format(42) ) #start: test 42 :end
print( 'start: test {:f} :end'.format(42) ) #start: test 42.000000 :end
print( 'start: test {:06.2f} :end'.format(3.141592) ) #start: test 003.14 :end
print( 'start: test {p[first]} {p[last]} :end'.format(p={'first': 'Jean-Luc', 'last': 'Picard'}) ) #start: test Jean-Luc Picard :end
print( 'start: test {p[4]} {p[5]} :end'.format(p=[4, 8, 15, 16, 23, 42]) ) #start: test 23 42 :end
print( 'start: test {:%Y-%m-%d %H:%M} :end'.format(dt.datetime(2001, 2, 3, 4, 5)) ) #start: test 2001-02-03 04:05 :end

### Search in files by Tags

In [233]:
import roamability as rb
reload(rb)

# Где искать файлы .txt с тегами
paths = ['c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_HOW_IT_WORKS/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/',
        'c:/W_DATA_ROAM/ПРОЕКТЫ/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_KNOWN_ISSUES/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/_NOC_M_NEW_CONNECTION/',
        'c:/W_DATA_ROAM/ALARMS/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES',
        ]

# Если хотя бы один из тэгов присутствует. Нужно указать минимум один.
# any_tags  = ['#reports_bss_usage_mop']
# any_tags  = ['#gy_principle_of_operation']
# any_tags  = ['#ocs_principle_of_operation']
# any_tags  = ['#reports_ocs_usage_mop']
# any_tags  = ['#reports_ocs_subscribers']
# any_tags = ['#epg_principle_of_operation']
any_tags  = ['#reports_ocs_subscribers']
# any_tags  = ['#pbx_principle_of_operation']

# Если все теги присутствуют. Нужно оставить лист пустым [] если нет обязательных тэгов.
all_tags = []

rb.find_files_by_tags(paths, all_tags, any_tags)

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\REPORT_OCS_Subscribers.txt
----------------------------------------------------------------------------------------------------
#reports_ocs_subscribers
------------------------------
Отчет выводит абонентов с привязкой к аккаунту и реселлеру

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\REPORT_Resellers_Accounts_Subscribers_A.txt
----------------------------------------------------------------------------------------------------
#reports_ocs_subscribers
------------------------------
Отчет Реселлер\Аккаунт\PP\Кол-во абонентов
Отчет Реселлер\Аккаунт\Абонент (IMSI, MSUSDN)\Prepaid package



### Search in ALARMS by folder names

In [577]:
import os

paths = [
         r'c:/W_DATA_ROAM/ALARMS/',
         r'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES/',
         r'c:/W_DATA_ROAM/ПРОЕКТЫ/',
         r'c:/W_DATA_ROAM/ПРОЕКТЫ_NEW_CONNECTION/',
    r'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ',
    r'c:\W_DATA_ROAM_DOC\ПРОЕКТЫ',
        ]

for path in paths:
    names = os.listdir(path)
    [print(normpath(path + name)) for name in names if name.upper().find('UKRUM')!=-1]

c:\W_DATA_ROAM\ALARMS\MOC_P4_UKRUM_190607_#3050591
c:\W_DATA_ROAM\ALARMS\PS_P4_UKRUM_190218
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\PRICES\P4_UKRUM.msg


### Search in Allowed Lists and others

In [534]:
downloads = 'C:/Users/balob/Downloads'
dmi_allowed_list = 'DMI.dmi_allowed_list_export_Thu_Jul_30_2020.csv'
dmi_netpfx = 'DMI.dmi_netpfx_export_Wed_Jun_03_2020.csv'
bss_usage = 'bss_usage_2019_08.csv' # c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\BSS_SQL\Report_BSS_Usage_per_PLMN.txt

#Like expression for Allowed List
df_al=pd.read_csv(join(downloads, dmi_allowed_list))
df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace={r'[Pp][Aa][rtner]?[\s\w-]*|^S1_[\s\w-]*':'S1',
                                                        r'[Pp]4[\s\w-]*|^S2_[\s\w-]*':'S2',
                                                        r'[Mm][Bb][\s\w-]*|^S4_[\s\w-]*':'S4',
                                                        r'SMART[\s\w-]*|^S5_[\s\w-]*':'S5',
                                                        r'^S8_[\s\w-]*':'S8'}, regex=True)
df_al.rename({'PLMN code':'TADIG'}, axis=1, inplace=True)

df_usage = pd.read_csv(join(downloads, bss_usage), sep=';')
cols = ['REP_DATE','VPLMN','SPONSOR','MOC','MOC_SMS','DATA']
df_usage = df_usage.pivot_table(values='COST', index=['REP_DATE', 'VPLMN', 'SPONSOR'], columns='USAGE_TYPE').reset_index()[cols]
df_usage.rename({'VPLMN':'TADIG'}, axis=1, inplace=True)
df_usage.columns.name=''

df_al = pd.merge(df_al, df_usage, how='left', on=['SPONSOR','TADIG'])

df_np=pd.read_csv(join(downloads, dmi_netpfx))

In [17]:
df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','REP_DATE','MOC','MOC_SMS','DATA','SPONSOR']].sort_values(by=['Sponsor','TADIG']).SPONSOR.unique()

array(['S2', 'S1'], dtype=object)

In [580]:
sponsor = 'Tottolli' # Use regular expressions if it's needed
tadig = 'TUR'

df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','REP_DATE','MOC','MOC_SMS','DATA','SPONSOR']].sort_values(by=['Sponsor','TADIG'])

,Sponsor,TADIG,REP_DATE,MOC,MOC_SMS,DATA,SPONSOR
4859,MB_Tottolli,TURTK,NaN,NaN,NaN,NaN,S4
5100,Partner_Tottolli,TURAC,NaN,NaN,NaN,NaN,S1
5030,Partner_Tottolli,TURTS,2019-08,138.629571,20.060967,4983.532942,S1
7049,S8_Tottolli,TURAC,NaN,NaN,NaN,NaN,S8
7050,S8_Tottolli,TURTK,NaN,NaN,NaN,NaN,S8
7051,S8_Tottolli,TURTS,NaN,NaN,NaN,NaN,S8


In [147]:
df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','SPONSOR']].sort_values(by=['Sponsor','TADIG']).Sponsor.unique()

array(['MB_DRIM01_ALL', 'P4_DRIM01_ALL', 'Partner_DRIM01_A',
       'S5_DRIM01_ALL'], dtype=object)

In [518]:
plmn = 'BLRMD'
df1 = df_al.loc[(df_al['TADIG'].str.startswith(plmn, na=False)) &
          (df_al['Sponsor'].str.contains('', na=False)), ['Sponsor','TADIG']].sort_values(by=['Sponsor','TADIG'])
df2 = df_np[(df_np.Operator.str.startswith(plmn,na=False))] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)
display(df1)
display(df2)

,Sponsor,TADIG
2049,PA_Telzar_Soft,BLRMD
10,Partner,BLRMD
1116,Partner_2018,BLRMD
2570,Partner_All,BLRMD
1792,Partner_Clay_Com,BLRMD
5636,Partner_Clementv,BLRMD
4020,Partner_Com_JWM,BLRMD
2265,Partner_Combined,BLRMD
4319,Partner_DIAL_COM,BLRMD
5388,Partner_DRIM01_A,BLRMD


,Operator,Prefix,Number/Range,SSN,MCC,MNC
238,BLRMD,375296,1,7,257,1
239,BLRMD,375296,1,149,257,1
240,BLRMD,375446,1,7,257,1
241,BLRMD,375446,1,149,257,1
1244,BLRMD,375293,1,7,257,1
1245,BLRMD,375293,1,149,257,1
4150,BLRMD,3752960000,0,7,257,1
4151,BLRMD,3752969999,0,7,257,1
4152,BLRMD,3754460000,0,7,257,1
4153,BLRMD,3754469999,0,7,257,1


In [816]:
str(df_np[(df_np.Prefix.str.startswith('393519999',na=False))].Prefix.unique().tolist())
df_np[(df_np.Prefix.str.startswith('3774',na=False))]

,Operator,Prefix,Number/Range,SSN,MCC,MNC
81,MCOM1,37744,1,7,212,1
82,MCOM1,37744,1,149,212,1
83,MCOM1,37745,1,149,212,1
1164,MCOM1,37745,1,7,212,1
1165,MCOM1,37744010405,0,1,212,1
5382,MCOM2,3774000,1,7,212,10
5383,MCOM2,3774000,1,149,212,10
5384,MCOM2,3774098,1,149,212,10
5385,MCOM2,3774098,1,7,212,10
5386,MCOM2,3774000400,1,1,212,10


### Hex Bin Dec conversion

In [593]:
len('0114010301001E01140000000300030501020000000000000000000000000000000000000000000000000000000000'.replace(' ', '')) / 2

47.0

In [3]:
hex_int = 0x0C
dec_int = 47
bin_int = 0b100
# Hex to dec
print('Hex to dec')
print(f'{hex_int:0>16d}')
# Hex to bin
print('Hex to bin')
print(f'{hex_int:0>16b}')
# Dec to hex
print('Dec to hex')
print(f'{dec_int:F>16X}')
# Dec to bin
print('Dec to bin')
print(f'{dec_int:0>16b}')
# Bin to hex
print('Bin to hex')
print(f'{bin_int:F>16X}')
# Bin to dec
print('Bin to dec')
print(f'{bin_int:0>16d}')

Hex to dec
0000000000000012
Hex to bin
0000000000001100
Dec to hex
FFFFFFFFFFFFFF2F
Dec to bin
0000000000101111
Bin to hex
FFFFFFFFFFFFFFF4
Bin to dec
0000000000000004


In [222]:
7-243-1
4-4-2

f'{bin(7)} - {bin(243)} - {bin(1)}'
#2.61.6

'0b111 - 0b11110011 - 0b1'

In [223]:
print(bin(10915))
a = '11111110011001'
# = '00000011001101'
print(int(a[-14:-11],2),'-',int(a[-11:-3],2),'-',int(a[-3:],2) )

0b10101010100011
7 - 243 - 1


### SIM Decoders

In [492]:
import codecs
codecs.encode(b"Smartel", "hex").upper()
spn = '547269706C6554656C' # 00547269706C6554656C9000 'TripleTel'
spn = '476C6F62616C3153494D' # 02476C6F62616C3153499000 'Global1SIM'
spn = '5469636F6D' # 005469636F6DFFFFFFFF9000 'Ticom'
spn = '006E6F722E6E6574'
spn = '524F414D4142494C49' # 2524F414D4142494C49
spn = '6E6F722E6E6574'
bytearray.fromhex(spn).decode()

'nor.net'

In [66]:
# IMSI coding
imsi = '2948310000209045'
imsi = '4952106941005106'
# imsi = '425010000000000'
# imsi = '4952100000000000'
# imsi = '234500000000000'
imsi = '2905201111111111'
def imsi_conv(imsi, code='code'):
    """Conver deconvert IMSI for perso. code='code' / 'decode'"""
    if code=='code':
        imsi = '9' + imsi
        return ''.join([imsi[i] for i in [1, 0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12, 15, 14]])
    elif code=='decode':
        return ''.join([imsi[i] for i in [0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12, 15, 14]])

imsi_conv(imsi, code='decode')

'250021111111111'

In [ ]:
2943050000000000

In [489]:
# ANY decoder
value = '98791232039009010707' # ICCID Ticom 987912320390090107079000
value = '07916437716243F0' # SPN Ticom 359999320001 FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFE1FFFFFFFFFFFFFFFFFFFFFFFF0791539999230010FFFFFFFF0000A89000
value = '049000006E6F722E6E6574'
def any_decoder(value):
    value_conv = ''
    for i in range(1, len(value), 2):
        value_conv += value[i: i + 1]
        value_conv += value[i - 1: i]
    return value_conv
any_decoder(value)
# 8997219121000046000
# 89972123300990746000
# 899721233009907460008997212330099074609F356239661D90F3D7

'40090000E6F627E2E65647'

## Testing

In [581]:
import smpplib
import smpplib.gsm
client = None
#parts, encoding_flag, msg_type_flag = smpplib.gsm.make_parts(u'Hello world!')
parts, encoding_flag, msg_type_flag = smpplib.gsm.make_parts("""
Test message.
""")

# client = smpplib.client.Client('172.18.11.20', 2770) # Prod
client = smpplib.client.Client('172.18.11.90', 2770) # Lab

client.connect()
# client.bind_transceiver(system_id='test', password='test') # Prod
client.bind_transceiver(system_id='test', password='ChangeMe') # Lab

for part in parts:
    pdu = client.send_message(
        source_addr_ton=smpplib.consts.SMPP_TON_INTL,
#         source_addr_npi=smpplib.consts.SMPP_NPI_ISDN,
        # Make sure it is a byte string, not unicode:
        source_addr='79217428080',
#         source_addr='IsraelPostIsraelPost',
        dest_addr_ton=smpplib.consts.SMPP_TON_INTL,
        #dest_addr_npi=smpplib.consts.SMPP_NPI_ISDN,
        # Make sure thease two params are byte strings, not unicode:
#         destination_addr='972559300036',
        destination_addr='79217428080',
        short_message=part,
        data_coding=encoding_flag,
        esm_class=msg_type_flag,
        registered_delivery=True,)
    print(f'PDU SQN: {pdu.sequence}')
client.unbind()
client.unbind()
client.disconnect()

PDU SQN: 2


In [547]:
len('IsraelPostIsraelPost')

20

In [747]:
import requests
ip_addr = '192.168.1.44'
port = 80
url_01 = f'http://{ip_addr}:{port}/climate;sensor=1&readattempt=10&temperature=25.0&humidity=35.0'
url_02 = f'http://{ip_addr}:{port}/climate;sensor=4&readattempt=10&temperature=25.0&humidity=35.0'
url_03 = f'http://{ip_addr}:{port}/gas;sensor=2&sensorValue=666'
url_04 = f'http://{ip_addr}:{port}/motion;sensor=3'
for url in [url_01, url_02, url_03, url_04]:
    r = requests.get(url)
    print(r.status_code)
    print(r.text.strip())

200
"OK"
200
"OK"
200
"OK"
200
"OK"


In [578]:
# Data
sdrRate = 1.405533
charge = 0.03440
chargedUnits = 3386368
unit = 1024 * 1014
(charge * sdrRate / chargedUnits) * unit

0.01482529177284548

In [458]:
# Sms
sdrRate = 1.37522
charge = 0.16527
chargedUnits = 1
unit = 1
(charge * sdrRate / chargedUnits) * unit

0.23303069999999998

In [460]:
# Voice
sdrRate = 1.405533
charge = 0.0282
chargedUnits = 106
unit = 60
(charge * sdrRate / chargedUnits) * unit

0.02243548901886792

In [1027]:
sql_srt="""
SELECT 
DISTINCT o.Name,did.Description,did.RangeStart, did.RangeEnd, did.Count
FROM [dbo].[DIDRanges] did
LEFT JOIN [dbo].[SponsorDIDRanges] sdid ON did.ID = sdid.DIDRangeID
LEFT JOIN [dbo].[Sponsors] s ON sdid.SponsorID = s.ID
LEFT JOIN [dbo].[Operators] o ON o.ID = s.OperatorID
WHERE o.Name LIKE 'X2%';"""
with rb.MssqlConnect('172.19.11.59', 'RoamDb', 'roamdb', 'roamdbpasswd') as cnxn:
    df_did_rdb = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_did_rdb[['RangeStart','RangeEnd']] = df_did_rdb[['RangeStart','RangeEnd']].astype('str')
df_did_rdb.drop_duplicates(inplace=True)

def f(x):
    l = []
    if 'UK' in x.Description:
        l.append('UK')
    elif 'USA' in x.Description:
        l.append('USA')
    else:
        l.append('Unk')
#     if '200311' in x.Description:
#         l.append('NEW_200311')
#     elif '200303' in x.Description:
#         l.append('NEW_200303')
#     else:
#         l.append('OLD')
    if '200311' in x.Description:
        l.append('NEW')
    elif '200303' in x.Description:
        l.append('NEW')
    else:
        l.append('OLD')
    if 'drimsim' in x.Description.lower():
        l.append('DrimSim')
    else:
        l.append(x.Description.split(' ')[-1])
    return l

df_did_rdb[['Country', 'Type', 'Customer']] = DataFrame(df_did_rdb.apply(f, axis=1).tolist())
df_did_rdb.head(3)

,Name,Description,RangeStart,RangeEnd,Count,Country,Type,Customer
0,X2one,X2One USA 200303 DrimSim,16282710001,16282710001,1,USA,NEW,DrimSim
1,X2one,X2One USA 200303 DrimSim,16282710002,16282710002,1,USA,NEW,DrimSim
2,X2one,X2One USA 200303 DrimSim,16282710003,16282710003,1,USA,NEW,DrimSim


In [1028]:
df_did_rdb.groupby(['Name','Country','Customer','Type'], as_index=False)['Count'].sum().sort_values(by='Type')

,Name,Country,Customer,Type,Count
0,X2one,USA,DrimSim,NEW,6000
1,X2one,USA,FREE,NEW,997
2,X2one,USA,OrangeTest,NEW,1
3,X2one,USA,Pavel,NEW,1
4,X2one,USA,test,NEW,1


In [1026]:
df_did_rdb.loc[df_did_rdb.RangeStart == '16672506138']
# df_did_rdb.loc[df_did_rdb.Customer == 'test']

,Name,Description,RangeStart,RangeEnd,Count,Country,Type,Customer
0,X2one,X2One USA 200303 1OT,16672506138,16672506138,1,USA,NEW,1OT


In [987]:
df_did_rdb.loc[df_did_rdb.Type == 'OLD',
               ['Description','RangeStart','RangeEnd','Count']].to_csv(join(downloads, 'X2oneOldDid.csv'), index=False)

In [1022]:
# df = pd.read_csv(join(downloads, 'Roamability_Drimsim_5K_replacement_200311.csv'),
df = pd.read_csv(join(downloads, 'US_Order_1_1k.csv'),
                 header=None, names=['RangeStart'], dtype='str')
df['DS'] = 'YES'
df.head()

,RangeStart,DS
0,16282710001,YES
1,16282710002,YES
2,16282710003,YES
3,16282710004,YES
4,16282710005,YES


In [1025]:
df1 = pd.merge(df_did_rdb.loc[df_did_rdb.Customer == 'DrimSim'], df, how='right', on='RangeStart')
df1.count()
df1.loc[df1.Customer.isna()]

,Name,Description,RangeStart,RangeEnd,Count,Country,Type,Customer,DS
999,NaN,NaN,16672506138,NaN,NaN,NaN,NaN,NaN,YES


In [434]:
ota_payload = '20603D79FB432EAE63F082E734F3B46085'
l = len(ota_payload)/2
as_rp = '0D0001'
as_mo = '0D0021'
sms_payload = f'02700000{int(l+5):X}{as_rp}0000{ota_payload}'
print(sms_payload)

02700000160D0001000020603D79FB432EAE63F082E734F3B46085


In [487]:
from Crypto.Cipher import DES3
from Crypto.Random import get_random_bytes
# Avoid Option 3
while True:
    try:
        key = DES3.adjust_key_parity(get_random_bytes(24))
        break
    except ValueError:
        pass

hex_str = 'D0AEE50D70087C19D773696FE698C78832432DF14F2487D1'
key = bytes.fromhex(hex_str)    

cipher = DES3.new(key, DES3.MODE_CBC)
# plaintext = b'We are no longer the knights who say ni!'
plaintext = bytes.fromhex('4E3F0E5A000000010018BC') 
plaintext = bytes.fromhex('4E3F0E5A00000001') 
msg = cipher.iv + cipher.encrypt(plaintext)
s=''
for b in msg:
    s+=f'{b:X}'
s
# 20603D79FB432EAE63F082E734F3B46085

'4155A1741F3A507AC4CFC395C16D9795'

### OTA messages

In [183]:
# Encrypt plaintext to OTA payload

import math
from Crypto.Cipher import DES3

# Roamability Test keys example

key_hex_str = 'D0AEE50D70087C19D773696FE698C78832432DF14F2487D1'
key = bytes.fromhex(key_hex_str)

# Plaintext

plaintext = '00A4000C023F0000A4000C027F2000A4000C026F4600B0000011'
print(f'The original plaintext: {plaintext}')

# Split plaintext into 8 byte parts for DES3 CBC encryption

byte_array_list = []
for i in range(1, math.ceil(len(plaintext)/16)+1):
    hex_str = f'{plaintext[(i-1)*16:i*16]:0<16}'
    print(f'Plaintext 8 byte part {i}: {hex_str}')
    byte_array = bytes.fromhex(hex_str)
    byte_array_list.append(byte_array)

# Create cipher

cipher = DES3.new(key, DES3.MODE_CBC)

# Encrypt 8 byte parts

cipher_str = ''
for byte_array in byte_array_list:
    msg = cipher.encrypt(byte_array_list[0])
    for b in msg:
        cipher_str += f'{b:X}'
print(f'Encrypted:        {cipher_str}')
print('Original example: 2028D0FC11B4CD2E540704D7775B065DA8')

The original plaintext: 00A4000C023F0000A4000C027F2000A4000C026F4600B0000011
Plaintext 8 byte part 1: 00A4000C023F0000
Plaintext 8 byte part 2: A4000C027F2000A4
Plaintext 8 byte part 3: 000C026F4600B000
Plaintext 8 byte part 4: 0011000000000000
Encrypted:        DE3D6717AC48A02B2C4BCF92B8FCFE28F97879F2E11970CCABED9F711D4A94
Original example: 2028D0FC11B4CD2E540704D7775B065DA8


In [675]:
checksum = 0
packet = bytes.fromhex('2028D0FC11B4CD2E540704D7775B065D')
for el in packet:
    checksum ^= el

print(hex(checksum))

0x95


In [685]:
# Make payload for SMS
ota_payload = '8010000016FBF7FF7F1F0E0FCB4100001FE3869804070000000000'

as_rp = '0D0001'
as_mo = '0D0021'
header = '0000BFFF010000000000'
l = (len(ota_payload) + len(as_rp) + len(header))/2
sms_payload = f'02700000{int(l):X}{as_rp}0000BFFF010000000000{ota_payload}'
print(sms_payload)

02700000280D00010000BFFF0100000000008010000016FBF7FF7F1F0E0FCB4100001FE3869804070000000000


In [185]:
from Crypto.Cipher import DES3
from Crypto.Random import get_random_bytes
# Avoid Option 3
while True:
    try:
        key = DES3.adjust_key_parity(get_random_bytes(24))
        break
    except ValueError:
        pass

cipher = DES3.new(key, DES3.MODE_CFB)
plaintext = b'We are no longer the knights who say ni!'
msg = cipher.iv + cipher.encrypt(plaintext)
msg

b"X{BK\xc3n\xf1P\x92\xe8\xa7\x02\x7f\xacM\x80\x85\x9b'\x84\xcc\x88\x10-\xbfM\xd8\xd9:J\xb1\x86\xa2?V\x99\x94\xebb2\x90c\x0b\xe6\x8e@\xde\xe1"

In [195]:
cipher = DES3.new(key, DES3.MODE_CBC)
cipher.encrypt(plaintext)

b'\x95v\x14-[\x89\xb2uW\x8b\xc6\xf1\xe9-\xdbB\nB\x8bu\xbe\x1b5\x17\xac0\xd0\xfd+\x7f\xae\x1eF\x8f\xc6\xc3\x89}\xcdd'

### Developments

In [444]:
hex_int = 0xFA
dec_int = 130
bin_int = 0b100
# Hex to dec
print('Hex to dec')
print(f'{hex_int:0>16d}')
# Hex to bin
print('Hex to bin')
print(f'{hex_int:0>16b}')
# Dec to hex
print('Dec to hex')
print(f'{dec_int:F>16X}')
# Dec to bin
print('Dec to bin')
print(f'{dec_int:0>16b}')
# Bin to hex
print('Bin to hex')
print(f'{bin_int:F>16X}')
# Bin to dec
print('Bin to dec')
print(f'{bin_int:0>16d}')

Hex to dec
0000000000000250
Hex to bin
0000000011111010
Dec to hex
FFFFFFFFFFFFFF82
Dec to bin
0000000010000010
Bin to hex
FFFFFFFFFFFFFFF4
Bin to dec
0000000000000004


In [318]:
32 * 3

96

In [503]:
l = len('42F004C08042F020C00042F070C00042F010C00042F042C0C042F030C08042F070008042F01000C0FFFFFF0000FFFFFF0000') / 2
print('Dec to hex')
print(f'{int(l)} -> 0x{int(l):X}')

Dec to hex
50 -> 0x32


In [507]:
ota_env =\
"""
D1 51 82 02  83 81 06 04   01 99 99 99  8B 45 40 06   
01 88 88 88  7F F6 02 70   20 22 23 11  30 35 02 70   
00 00 30 15  16 01 19 19   B0 00 11 55  2C 28 B7 60   
04 80 D0 33  56 08 78 29   5A F4 7B 5C  12 A1 7A 00   
73 9B 60 A3  CB A0 58 4F   B5 03 61 B9  6C 38 B7 47   
F0 13 17 
"""

def ota_cmd(ota_env):
    ota_env = ota_env.replace(' ', '').replace('\n', '')
    print(f'{ota_env}')
    print(f'send_apdu -APDU A0C20000{int(len(ota_env)/2):X}{ota_env}')

ota_cmd(ota_env)

D151820283810604019999998B454006018888887FF6027020222311303502700000301516011919B00011552C28B7600480D033560878295AF47B5C12A17A00739B60A3CBA0584FB50361B96C38B747F01317
send_apdu -APDU A0C2000053D151820283810604019999998B454006018888887FF6027020222311303502700000301516011919B00011552C28B7600480D033560878295AF47B5C12A17A00739B60A3CBA0584FB50361B96C38B747F01317


In [511]:
s1 = 'D151820283810604918888888B454006919999997FF6027020222311303502700000301516011919B000114C98D1706B7EAF49335514A6463B8939518A4CB6689E09C0A90861AF60EB0060AE93927804C6E810'
s2 = 'D151820283818604918888888B454006919999997FF6000000000000003502700000301516011919B000114C98D1706B7EAF49335514A6463B8939518A4CB6689E09C0A90861AF60EB0060AE93927804C6E810'
s1 == s2

False

In [392]:
data_size = 8389594
time = 1
(data_size / time) / (1024 * 1024)

8.000940322875977

In [478]:
config = '010301001E001000000003000305010202000000'
config = '010301001E001000000003000305010202000000'
# 00077FF62902710000240ABFFF01000000000000000000005B00000300001E001400000003000305010201000000
# config = '010301001E011400000003000305010200000000' # default profile order  01 00
# config = '010300001E011400000003000305010201000000' # order 00 01

applet_enabling_flag = config[0:2]
applet_enabling_flag_dict = {'01':'Activate','00':'Deactivate'}

mode = config[2:4]
mode_dict = {'01':'Events only mode','02':'Timer only mode','03':'Events & Timer mode'}

default_imsi = config[4:6]
default_imsi_dict = {'00':'IMSI0','01':'IMSI1','02':'IMSI2','03':'IMSI3','04':'IMSI4','05':'IMSI5'}

polling_period = config[6:10]

features_activation_flags = config[10:14]

scan_delay_timer = config[22:26]

print(f'Applet enabling flag: {applet_enabling_flag_dict[applet_enabling_flag]}')
print(f'Mode: {mode_dict[mode]}')
print(f'Default IMSI: {default_imsi_dict[default_imsi]}')
print(f'Polling period: {int(polling_period, 16)} sec.')
print(f'Features activation flags: {int(features_activation_flags, 16):0>16b}')
# Features activation flags 1(1 Enable, 0 Disable):
# b1:Start with Primary (После рестарта UE будет выбран профайл 0).
# b2:Reset Loci, PS Loci (обнуление TIMSI).
# b3:RB_MODE (В HOME MCC будут перебираться IMSI спонсора).
# b4 Skip Primary IMSI in roaming: (Исключение Home IMSI в роуминге).
# b5:
# b6-b16: RFU
print(f'Scan delay timer: {int(scan_delay_timer, 16)} sec.')


Applet enabling flag: Activate
Mode: Events & Timer mode
Default IMSI: IMSI1
Polling period: 30 sec.
Features activation flags: 0000000000010000
Scan delay timer: 3 sec.


In [509]:
'027000001F0D00010000BFFF01000000000000102F978372C730834EF445172BD26B8C8F' # get_info
'027000001F0D00210000BFFF01000000000000102F978372C730834EF445172BD26B8C8F' # get_info_mo
'027000001F0D00010000BFFF0100000000000010733FE1C20ECA937911B9067460AA6612' # get_config
'027000001F0D00210000BFFF0100000000000010733FE1C20ECA937911B9067460AA6612' # get_config_mo
get_config = '027000001F0D00010000BFFF0100000000000010733FE1C20ECA937911B9067460AA6612'

get_EFHPPLMN = '02700000301516011919B00011552C28B7600480D033560878295AF47B5C12A17A00739B60A3CBA0584FB50361B96C38B747F01317'

# soup = BeautifulSoup(rb.sendSMS('97254120624', '425019628115014', '46705012099', get_EFHPPLMN),'xml')
soup = BeautifulSoup(rb.sendSMS('97254120624', '425019613000536', '79219600300', get_EFHPPLMN),'xml')
soup

<?xml version="1.0" encoding="utf-8"?>
<response>
<result>0</result><response>40077FF64502710000400AB000110000000009000004900042F004C08042F020C00042F070C00042F010C00042F042C0C042F030C08042F070008042F01000C0FFFFFF0000FFFFFF0000</response>
<src_gt>46705009041</src_gt>
</response>

In [ ]:
%reset -f